In [1]:
import pandas as pd

# complete data-range
date0 = pd.to_datetime('2010-01-04')
date1 = pd.to_datetime('2024-11-30')
future = pd.to_datetime('2030-05-16')

def canonical_columns(df):
    df = df.copy()
    df['date'] = pd.to_datetime(df['time'], unit='s')
    df['date'] = pd.to_datetime(df['date'].dt.date)
    df = df[['date', 'open']]
    df = df.rename(columns={'open': 'usd'})
    return df
 
def extend_dates(df, start_date, end_date):
    # extend by future dates
    date_range = pd.date_range(start=start_date, end=end_date, freq='D')
    df = pd.merge(df, pd.DataFrame({'date': date_range}), on='date', how='right')
    df.sort_values('date', inplace=True)

    # Use forward fill (ffill) to fill missing values with the last known value
    df['usd'] = df['usd'].ffill()
    return df

#btc = pd.read_csv('data/raw/BTCUSD.csv')
btc = pd.read_csv('data/BTCUSD_2024_11_21.csv')
btc = canonical_columns(btc)
btc = extend_dates(btc, date0, future)

# Set the condition to identify rows with dates larger than today
condition = btc['date'] > date1

# Erase data in the 'DataColumn' for dates larger than today
btc.loc[condition, 'usd'] = None  # Replace with None or any other value you prefer

display(btc)

,date,usd
0,2010-01-04,0.000672
1,2010-01-05,0.000672
2,2010-01-06,0.000672
3,2010-01-07,0.000672
4,2010-01-08,0.000672
...,...,...
7433,2030-05-12,NaN
7434,2030-05-13,NaN
7435,2030-05-14,NaN
7436,2030-05-15,NaN


In [2]:
import numpy as np

def _unix_to_btc_time(date):
    first = 1.2625632e+09
    delta = 1e7
    return date - first + delta

def _bitcoin_time(date):
    x = pd.to_datetime(date).copy()
    x = x.apply(lambda x: x.timestamp())
    return _unix_to_btc_time(x)
    
def _evaluate_fit(fit, date):
    # date -> btc-time
    x = pd.to_datetime(date)
    x = x.apply(lambda x: x.timestamp())
    x = _unix_to_btc_time(x)
    x = np.log(x)
    return np.exp(fit[0] * x + fit[1])

def btc_fit(dates, price):
    """
    Fit to 
    """
    # logscale linear fit
    t = _bitcoin_time(dates)
    t = np.log(t)
    y = np.log(price)
    btc_fit = np.polyfit(t, y, 1)
    return _evaluate_fit(btc_fit, btc['date'])

In [3]:
def log_fits(btc, last_included_date):
    # Create a copy of the original DataFrame
    df = btc.copy()
    # Limit the DataFrame to the specified date range
    df = df[df['date'] < pd.to_datetime(last_included_date)].copy()

    # Compute the main fit for the filtered range
    btc['fit'] = btc_fit(df['date'], df['usd'])

    # Compute undervalued fit
    btc_undervalued = df[df['usd'] < btc.loc[df.index, 'fit']]  # Mask only for relevant indices
    btc['undervalued'] = btc_fit(btc_undervalued['date'], btc_undervalued['usd'])

    # Compute overvalued fit
    btc_overvalued = df[df['usd'] > btc.loc[df.index, 'fit']]  # Mask only for relevant indices
    btc['overvalued'] = btc_fit(btc_overvalued['date'], btc_overvalued['usd'])

    # Compute bubble fit
    btc_bubble = df[df['usd'] > btc.loc[df.index, 'overvalued']]  # Mask only for relevant indices
    btc['bubble'] = btc_fit(btc_bubble['date'], btc_bubble['usd'])

    # Compute top fit
    btc_top = df[df['usd'] > btc.loc[df.index, 'bubble']]  # Mask only for relevant indices
    btc['top'] = btc_fit(btc_top['date'], btc_top['usd'])

    return btc

# Example usage
today = '2024-11-30'
btc = log_fits(btc, last_included_date='2024-11-30')

display(btc)


,date,usd,fit,undervalued,overvalued,bubble,top
0,2010-01-04,0.000672,0.000280,0.000107,0.000806,0.002362,0.005253
1,2010-01-05,0.000672,0.000292,0.000111,0.000840,0.002458,0.005459
2,2010-01-06,0.000672,0.000305,0.000116,0.000876,0.002558,0.005671
3,2010-01-07,0.000672,0.000318,0.000122,0.000913,0.002660,0.005890
4,2010-01-08,0.000672,0.000331,0.000127,0.000950,0.002766,0.006115
...,...,...,...,...,...,...,...
7433,2030-05-12,NaN,312323.305544,219546.996280,505483.835998,620783.989059,696530.608310
7434,2030-05-13,NaN,312529.680497,219696.329274,505808.596086,621165.698736,696943.778594
7435,2030-05-14,NaN,312736.164457,219845.744043,506133.521770,621547.592518,697357.139192
7436,2030-05-15,NaN,312942.757466,219995.240622,506458.613111,621929.670468,697770.690167


In [8]:
# Risk
v0 = np.log(btc['undervalued'])
v1 = np.log(btc['top'])
btc['risk'] = (np.log(btc['usd']) - v0) / (v1 - v0)

v0 = btc['undervalued']
v1 = btc['top']
btc['lin_risk'] = (btc['usd'] - v0) / (v1 - v0)
display(btc[['risk', 'lin_risk']])

,risk,lin_risk
0,0.472374,0.109882
1,0.461711,0.104842
2,0.451108,0.100028
3,0.440564,0.095427
4,0.430080,0.091028
...,...,...
7433,NaN,NaN
7434,NaN,NaN
7435,NaN,NaN
7436,NaN,NaN


In [9]:
import plotly.express as px
fig = px.line(btc, x='date', y=['risk'],
              title='Historical Bitcoin Price and Logarithmic Regression',
              labels={'date': 'Date', 'open-price': 'Open Price', 
                      'fit': 'Fit', 
                      'undervalued': 'Undervalued',
                      'overvalued': 'Overvalued',
                      'top': 'Bubble Top'})
fig.update_traces(mode='lines', hovertemplate='%{y}')
fig.update_layout(hovermode='x unified')
fig.show()

In [11]:
from bitcoin_risk.plotter import bitcoin_plot
fig = bitcoin_plot(btc)
fig.write_html('btc.html')
fig.show()
